# Few shot learning approach to stressful events



In [ ]:
### helper to check memory on GPU
import os
import gc
os.environ['CUDA_VISIBLE_DEVICES'] = '0' ### select GPU 1 
import subprocess as sp

def get_gpu_memory(): ### function to check memory
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

get_gpu_memory()[0]

In [ ]:
!pip install -U transformers  ### requires newest transformers
!pip install torch
!pip install huggingface_hub
!pip install accelerate
!pip install bitsandbytes
!pip install scipy
!pip install guidance

In [ ]:
from huggingface_hub import notebook_login,login
login(token='')  ### use token from HF or use notebook_login method 

In [ ]:
import torch
from transformers import AutoModel, BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
from guidance import models, gen, select
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device ### Ensure that device type = 'cuda' below, otherwise your session is not GPU enabled.

In [ ]:
!pip install -U autoawq

In [ ]:
#!pip install -U transformers
#!export FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE 
#!pip install -U flash-attn
## make sure transformers >= 4.37.2
import transformers; print(transformers.__version__)
#import autoawq; print(autoawq.__version__)

In [ ]:
#!pip install -U git+https://github.com/huggingface/transformers.git@fdb85be40fa255c015819e711c15117c2aaa5101
# Specify the model name
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1" 
#model_name = "TheBloke/Mixtral-8x7B-Instruct-v0.1-AWQ" 
# quantization settings 
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/home/scherbak/model")
double_quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=False)
# load model
mistral = models.Transformers(model_name,attn_implementation="flash_attention_2", cache_dir="/home/scherbak",quantization_config=double_quant_config)#,device_map = 'cuda')
#tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mixtral-8x7B-Instruct-v0.1-AWQ")
#model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir="/home/scherbak/model")
#mistral = models.Transformers(model_name,cache_dir="/home/scherbak/model",device_map = 'cuda')

#model = AutoModelForCausalLM.from_pretrained(model_name,device_map = 'cuda')

In [ ]:
### csv file with social history column 

import pandas as pd
df = pd.read_csv('SDOH_SPARC21093/social.matched.csv')
social = df.social 


In [ ]:
len(social)
#get_gpu_memory()[0] ### check GPU free memory

In [ ]:
answers=[]
import re
#myvar=myvar[473:]
i=0
for text in social[i:]:
    print (i)
    i=i+1
    answer=mistral +  f'''
    Read a question with short description of patient's life situation. Then choose the best answer.
    ## Question:
    Pt is divorced. He is a carpenter.
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
    It is mentioned that patient is divorced but nothing indicates that divorce happened after 2012. Hence no answer is correct. 
    ## Final answer: 21.
    
    ## Question:
    Pt is living in [ Name ] with her mother. She has a boyfriend who she sees on weekends. No smoking and alcohol as per patient. She has 2 children from previous marriage which just terminated. No known history of illicit drug usage. 
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
    It is mentioned that patient's marriage has recently terminated. Hence answer 7 is correct. 
    ## Final answer: 7.

    ## Question:
    Pt quit smoking. She is separated from husband. She denies doing drugs. Currently on medication. Never drank alcohol. 
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
    Non of the life events from the list are mentioned in the question text. It is mentioned patient is separated, but there is no indication it happened recently. Hence answer 21 is correct. 
    ## Final answer: 21.
    
    ## Question: 
    Pt is married and has 2 children from prior marriage. He lost his job in 2020. Currently lives with his dad in [ Name ]. He has no girlfriend other than his spouse.
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
    It is mentioned that patient lost his job and it happened recently (in 2020). Hence answer 4 is correct. 
    ## Final answer: 4.
    
    ## Question: 
    Pt lives alone in [home]. He has no children. He is an immigrant from Bolivia. He is often called slurs at his job based on his skin color
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
    It is mentioned that patient is an immigrant, but it is not mentioned that it was a forced move. It is mentioned that he is often being discriminated at work based on skin color.
    Hence answer 13 is correct. 
    ## Final answer: 13.
    
    ## Question:
    {text[:350]}
    Which of the following life events has likely happened to the person recently. Recently means either mentioned that it is recent or just happened, or it is mentioned that it happened on or after 2012. 
    Select only one best applicable answer:
    1: somebody close to the patient died, but not their spouse
    2: was evicted from home or otherwise was forced out of their home
    3: immigration or displacement due to war, political upheaval or other adverse circumstances
    4: was fired from a job or was laid off
    5: got arrested or sent to jail
    6: got recently pregnant
    7: got recently divorced
    8: got recently separated from a spouse or long-term partner
    9: long-term relationship recently broke down (not divorce)
    10: became unemployed
    11: lost financial means to support themselves or was bankrupt
    12: was recently critically injuried or diagnosed with life-threatening health condition
    13: experienced an event of discrimination based on sex, gender identity, race, sexual orientation or minority status
    14: was a victim of assault
    15: gone through suicidial attempt
    16: experienced abuse as a child or lived in problematic household
    17: spouse died
    18: became homeless
    19: got recently married or started living with partner
    20: started dating a boyfriend or girlfriend
    21: no applicable answer
    ## Explanation:
''' + gen(stop='Final answer',temperature=0.5,max_tokens=300)
    ans=re.findall(r'\d+', str(answer))[-1]
    #input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
    #output = model.generate(inputs=input_ids, temperature=0.1, do_sample=True,max_new_tokens=300)
    #print(tokenizer.decode(output[0]))
    print(ans)
    #answers.append(ans['answer'])
    with open("/home/scherbak/SDOH_SPARC21093/predictions/2/"+str(i)+".txt", "w") as text_file:
        text_file.write(ans)
